In [1]:
import numpy as np 
import pandas as pd 
import re
data = pd.read_csv('edited_data.csv')
X = data['Objectives']+'. '+ data['Description']

X = X.str.replace('<ul>', '').str.replace('<li>','').str.replace('</li>','').str.replace('</ul>','').str.replace('\n','')

X = X.fillna('')

In [2]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
#nltk.download('punkt')
#nltk.download("stopwords")
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
stop_words = set(stopwords.words('english')) 
not_delete = ["but", "shan't", "wasn't", "couldn't", "didn't", "hadn't", "against", "no", "haven't", "shouldn't", "needn't", "wouldn't", "aren't", "mightn't", "won't", "isn't", "hasn't", "don't", "mustn't", "doesn't", "not"]
STOPWORDS = [w for w in stop_words if w not in not_delete]

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

X = X.apply(clean_text)
X = X.str.replace('\d+', '')
print(X)

0       contribution metu hs ankara lab protocols coll...
1       attend makerfaire hannover demonstrate first p...
2       organize biotech day present project large gro...
3       contact connect community labs europe visit ig...
4       invite technik garage german community lab aac...
                              ...                        
2148                                                     
2149     attended fourth conference china igemer commu...
2150     created week long exhibition three igem teams...
2151     host architects competition microorganism app...
2152     secure funding supplies metagenomic sequencin...
Length: 2153, dtype: object


In [3]:
porter = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

for i in range(0,X.count()):
    X[i] = stemSentence(X[i])

In [4]:
print(type(X))

<class 'pandas.core.series.Series'>


In [5]:
#multilabel classifier: project tag %95
from sklearn.preprocessing import MultiLabelBinarizer
from scipy import stats

y2 = data['Project Tag']
mlb = MultiLabelBinarizer()
y2 = y2.fillna("N/A")
y2 = y2.str.split(', ')

for i in range(0, len(y2)):
    for j in range(0, len(y2[i])):
        y2[i][j] = y2[i][j].replace("Community Event ", "Community Event")\
        .replace("Conference/Panel Discussion ", "Conference/Panel Discussion")\
        .replace("Educational Material ", "Educational Material")\
        .replace("Social Media ", "Social Media").replace("Survey ", "Survey")\
        .replace("Teaching Activity ", "Teaching Activity")
    
mlb_y2 = mlb.fit_transform(y2)

#print(mlb.classes_)

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, mlb_y2, test_size=0.3, random_state=52)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.feature_extraction.text import TfidfVectorizer

'''
#Bag of Words
OneVsRest_classifier = Pipeline([
('vectorizer', CountVectorizer(ngram_range=(1, 2),strip_accents = 'unicode')),
('tfidf', TfidfTransformer(sublinear_tf=True)),
('clf', OneVsRestClassifier(SVC(C=1.0, kernel='linear')))])

OneVsRest_classifier.fit(X_train, y_train) 
predicted = OneVsRest_classifier.predict(X_test)


#exact match
print(OneVsRest_classifier.score(X_test, y_test))

#simple match
print('OneVsRestClassifier_test:'+ str(np.mean(predicted == y_test)))
'''

Tfidf_vect = TfidfVectorizer(ngram_range=(1, 2), strip_accents = 'unicode')
#X_train = Tfidf_vect.fit_transform(X_train)
#X_test = Tfidf_vect.transform(X_test)
X_tran = Tfidf_vect.fit_transform(X)

# Fit an ensemble of SVM classifier chains and take the average prediction of all the chains.
from sklearn.metrics import jaccard_score
chains = [ClassifierChain(SVC(C=1.0, kernel='linear'), order='random', random_state=i)
          for i in range(10)]
for chain in chains:
    #chain.fit(X_train, y_train)
    chain.fit(X_tran, mlb_y2)
    
#Y_pred_chains = np.array([chain.predict(X_test) for chain in chains])


#0项是mode，1项是count
#y_pred = stats.mode(Y_pred_chains)[0]


#simple match
#print('Chain_Classifier_test:'+ str(np.mean(y_pred == y_test)))

In [6]:
#OneVsRest_classifier.fit(X, mlb_y2)
new_data = pd.read_csv('initial_data.csv')
X_new = new_data['Descriptions']
X_new = X_new.str.replace('\n', '').str.replace('\t','')
X_new = X_new.fillna('')

X_new_tran = Tfidf_vect.transform(X_new)

Y_pred_chains_new = np.array([chain.predict(X_new_tran) for chain in chains])
y_pred_new = stats.mode(Y_pred_chains_new)[0]

#predicted1 = OneVsRest_classifier.predict(X_new)

output = mlb.inverse_transform(y_pred_new[0])

ValueError: Expected indicator for 17 classes, but got 1096

In [17]:
output = mlb.inverse_transform(y_pred_new[0])